Current Floods	dis.005	http://www.gdacs.org/flooddetection/overview.aspx

dis.005 http://www.gdacs.org/flooddetection/overview.aspx
SRC: http://www.gdacs.org/flooddetection/DATA/ALL/
Time period: 2002 - now
File type: Tif
Notes: 
Includes its own API: http://www.gdacs.org/flooddetection/Download/Technical_Note_GFDS_Data_Products_v1.pdf
Several parts of their API have been deprecated, i.e. finding nearby population, river, cities, slope, agricultural and urban land use. Probably because of lack of updated data. We can re-implement this.
Ask Perry about how the avg/sd deviations are calculated - same place throughout a year? This seems most likely. We want to know where the water is away from overall average, not "how extreme" the event is compared to the past on that day.

SpecialTiffs
- oceanmask.tif
- dma_global, dma_global_WGS84_v2, and dma_global_WGS84_Shift_v2 tifs

4 day average tifs (single sensor)

AvgSourceTiffs (Tm)
- brightness temperature of last measurement for this pixel (unit: K)
AvgCalibrationTiffs (Tc)
- brightness temperature of last calibration pixel
AvgPositionTiffs (P)
- relative position of calibration pixel

Merged 4 day tifs (all sensors)
- each pixel calculated based on the most available samples from any satellite in the constellation

AvgSignalTiffs (s = Tm/Tc)
- average flood signal
AvgMagTiffs (unit: # of sd from mean)
- average flood magnitude
* Average and standard deviation are calculated using the entire time series


*** difference between GDFS and UMD Global Flood Monitoring System (GFMS) https://pmm.nasa.gov/TRMM/flood-and-landslide-monitoring



Import libraries

In [0]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3 tools

In [0]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/*"

s3_file = "*.tif"

s3_key_orig = s3_folder + s3_file
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"


class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Potentially useful for online directories

In [0]:
# View files in a ftp, see cli.015
remote_path = "ftp://cidportal.jrc.ec.europa.eu/jrc-opendata/EDGAR/datasets/v431_v2"
remote_path_BC = remote_path + "/BC/TOTALS/"
file = req.urlopen(remote_path_BC).read().splitlines()

# Copy from https or ftp
with(closing(urlopen(online_folder + most_recent))) as r:
    with(open(local_orig, 'wb')) as f:
        shutil.copyfileobj(r, f)

Define local file locations

In [0]:
local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/*"
file_name = "*"]

local_orig = local_folder + file_name

orig_extension_length = 4 #4 for each char in .tif
local_edit = local_orig[:-orig_extension_length] + "edit.tif" 

Use rasterio to reproject and compress

In [0]:
# Note - this is the core of Vizz's netcdf2tif function

with rio.open(local_orig, 'r') as src:
    # This assumes data is readable by rasterio
    # May need to open instead with netcdf4.Dataset, for example
   
    data = src.read()[0]
    
    rows = data.shape[0]
    columns = data.shape[1]
    
    print(rows)
    print(columns)
    
    # Latitude bounds
    south_lat = -90
    north_lat = 90

    # Longitude bounds
    west_lon = -180
    east_lon = 180
    
    transform = rasterio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)
    
    # Profile
    
    no_data_val = *
    target_projection = 'EPSG:4326'
    target_data_type = np.float64
    
    profile = {
        'driver':'GTiff', 
        'height':rows, 
        'width':columns, 
        'count':1, 
        'dtype':target_data_type, 
        'crs':target_projection, 
        'transform':transform, 
        'compress':'lzw', 
        'nodata': no_data_val
    }
    
    with rio.open(local_edit, "w", **profile) as dst:
        dst.write(data.astype(profile["dtype"]), 1)

Upload orig and edit files to s3

In [0]:
# Original
s3_upload.upload_file(local_orig, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_orig))

# Edit
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))